<a href="https://colab.research.google.com/github/PathakYash1/Classification-of-Questions-asked-on-SO/blob/main/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Drive Connect

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)


Mounted at /content/gdrive


Creating the IDF


In [ ]:
import pandas as pd
import re
df_idf=pd.read_csv("/content/gdrive/MyDrive/ASE/RP_DATA/StackSample 10percent of Stack Overflow Q&A/10K_Questions.csv")

def pre_process(text):
    
    # lowercase
    text=str(text)
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf['text'] = df_idf['Title'] + df_idf['Body']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the first 'text'
df_idf


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,text
0,80.0,26.0,2008-08-01T13:57:07Z,NaN,26.0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sqlstatement execute multiple queries in one s...
1,90.0,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144.0,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,good branching and merging tutorials for torto...
2,120.0,83.0,2008-08-01T15:50:08Z,NaN,21.0,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,asp net site maps has anyone got experience cr...
3,180.0,2089740.0,2008-08-01T18:42:19Z,NaN,53.0,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,function for creating color wheels this is som...
4,260.0,91.0,2008-08-01T23:22:08Z,NaN,49.0,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,adding scripting functionality to net applicat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("/content/gdrive/MyDrive/ASE/RP_DATA/StackSample 10percent of Stack Overflow Q&A/stopwords.txt")

#get the text column 
docs=df_idf['text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  % sorted(inconsistent)


In [ ]:
word_count_vector.shape


(1048575, 38285)

In [ ]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  % sorted(inconsistent)


(1048575, 10000)

In [ ]:
list(cv.vocabulary_.keys())[:10]


['sqlstatement',
 'execute',
 'multiple',
 'queries',
 'statement',
 've',
 'written',
 'database',
 'generation',
 'script']

In [ ]:
list(cv.get_feature_names())[2000:2015]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['datafiledatarow',
 'datagrid',
 'datagridtemplatecolumn',
 'datagridtextcolumn',
 'datagridview',
 'datagridviewcomboboxcolumn',
 'dataid',
 'dataitem',
 'datalist',
 'datamember',
 'dataobject',
 'dataobjecttype',
 'datareader',
 'datarow',
 'datarowview']

TfidfTransformer to Compute Inverse Document Frequency (IDF)


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [ ]:
tfidf_transformer.idf_


array([13.76433132, 13.07118414, 13.07118414, ..., 14.16979643,
       13.07118414, 14.16979643])

Computing TF-IDF and Extracting Keywords


In [ ]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_csv("/content/gdrive/MyDrive/ASE/RP_DATA/StackSample 10percent of Stack Overflow Q&A/10K_Questions.csv")

df_test['text'] = df_test['Title'] + df_test['Body']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_title=df_test['Title'].tolist()
docs_body=df_test['Body'].tolist()
#print(docs_test)
#print(docs_title)
#print(docs_body)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=5):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,5)

# now print the results
print("\n=====Title=====")
print(docs_title[0])
print("\n=====Body=====")
print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====
SQLStatement.execute() - multiple queries in one statement

=====Body=====
<p>I've written a database generation script in <a href="http://en.wikipedia.org/wiki/SQL">SQL</a> and want to execute it in my <a href="http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime">Adobe AIR</a> application:</p>

<pre><code>Create Table tRole (
      roleID integer Primary Key
      ,roleName varchar(40)
);
Create Table tFile (
    fileID integer Primary Key
    ,fileName varchar(50)
    ,fileDescription varchar(500)
    ,thumbnailID integer
    ,fileFormatID integer
    ,categoryID integer
    ,isFavorite boolean
    ,dateAdded date
    ,globalAccessCount integer
    ,lastAccessTime date
    ,downloadComplete boolean
    ,isNew boolean
    ,isSpotlight boolean
    ,duration varchar(30)
);
Create Table tCategory (
    categoryID integer Primary Key
    ,categoryName varchar(50)
    ,parent_categoryID integer
);
...
</code></pre>

<p>I execute this in Adobe AIR using the following meth

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,5)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [ ]:
idx=120
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
How do you kill all current connections to a SQL Server 2005 database?

=====Body=====
<p>I want to rename a database, but keep getting the error that 'couldn't get exclusive lock' on the database, which implies there is some connection(s) still active.</p>

<p>How can I kill all the connections to the database so that I can rename it?</p>


===Keywords===
database 0.535
rename 0.381
kill 0.371
connections 0.348
implies 0.208


Generate keywords for a batch of documents


In [ ]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,5)
    
    if keywords != {}:
      results.append(keywords)

    

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
df = df.dropna(how='any',axis=0)
df

,doc,keywords
0,sqlstatement execute multiple queries in one s...,"{'sqlstatement': 0.441, 'integer': 0.365, 'sql..."
1,good branching and merging tutorials for torto...,"{'tortoisesvn': 0.467, 'branching': 0.467, 'me..."
2,asp net site maps has anyone got experience cr...,"{'site': 0.375, 'asp': 0.353, 'net': 0.321, 'n..."
3,function for creating color wheels this is som...,"{'pseudo': 0.303, 'colors': 0.295, 'solved': 0..."
4,adding scripting functionality to net applicat...,"{'card': 0.603, 'cards': 0.28, 'game': 0.244, ..."
...,...,...
9995,geographic location uri scheme i d like to use...,"{'uri': 0.723, 'scheme': 0.359, 'geo': 0.313, ..."
9996,how to send signal from one program to another...,"{'signal': 0.577, 'program': 0.427, 'queue': 0..."
9997,how to conditionally compile vc resources depe...,"{'_debug': 0.424, 'compile': 0.368, 'condition..."
9998,getprivateprofilestring bug i encrypted some t...,"{'ini': 0.518, 'file': 0.326, 'bug': 0.291, 's..."


In [ ]:
lst=df['keywords']
lst=lst.values.tolist()
lst

[{'integer': 0.365,
  'sqlconnection': 0.325,
  'sqlstatement': 0.441,
  'strsql': 0.239,
  'varchar': 0.241},
 {'branching': 0.467,
  'explaining': 0.231,
  'merging': 0.455,
  'tortoisesvn': 0.467,
  'tutorials': 0.409},
 {'asp': 0.353, 'need': 0.209, 'net': 0.321, 'site': 0.375, 'sitemap': 0.18},
 {'color': 0.266,
  'colors': 0.295,
  'pseudo': 0.303,
  'solved': 0.282,
  'stuck': 0.275},
 {'card': 0.603,
  'cards': 0.28,
  'class': 0.237,
  'database': 0.186,
  'game': 0.244},
 {'classes': 0.539,
  'mine': 0.193,
  'nested': 0.278,
  'programmers': 0.185,
  'video': 0.276},
 {'consume': 0.209,
  'consumption': 0.222,
  'services': 0.327,
  've': 0.259,
  'web': 0.411},
 {'database': 0.211,
  'given': 0.174,
  'live': 0.405,
  'server': 0.249,
  'sql': 0.371},
 {'build': 0.191,
  'incremented': 0.267,
  'number': 0.274,
  'settings': 0.315,
  'version': 0.465},
 {'hkcu': 0.211,
  'project': 0.251,
  'registry': 0.338,
  'setup': 0.219,
  'user': 0.477},
 {'connect': 0.541,
  'databa

In [ ]:
import operator
def Convert(tup, di):
    for a, b in tup:
        di.setdefault(a, []).append(b)
    return di

nlst=[]
for i in lst:
  d=i
  per_dict= {}
  sorted_d = sorted(d.items(), key=lambda x: x[1], reverse=True)
  #print(sorted_d)
  x=Convert(sorted_d, per_dict)
  #td={}
  #for j in sorted_d:
    #td[j[0]]=j[1]  
  nlst.append(x)
  #print(td)
print(nlst)
for i in nlst:
  for j in i.keys():
    i[j]
nlst

[{'sqlstatement': [0.441], 'integer': [0.365], 'sqlconnection': [0.325], 'varchar': [0.241], 'strsql': [0.239]}, {'tortoisesvn': [0.467], 'branching': [0.467], 'merging': [0.455], 'tutorials': [0.409], 'explaining': [0.231]}, {'site': [0.375], 'asp': [0.353], 'net': [0.321], 'need': [0.209], 'sitemap': [0.18]}, {'pseudo': [0.303], 'colors': [0.295], 'solved': [0.282], 'stuck': [0.275], 'color': [0.266]}, {'card': [0.603], 'cards': [0.28], 'game': [0.244], 'class': [0.237], 'database': [0.186]}, {'classes': [0.539], 'nested': [0.278], 'video': [0.276], 'mine': [0.193], 'programmers': [0.185]}, {'web': [0.411], 'services': [0.327], 've': [0.259], 'consumption': [0.222], 'consume': [0.209]}, {'live': [0.405], 'sql': [0.371], 'server': [0.249], 'database': [0.211], 'given': [0.174]}, {'version': [0.465], 'settings': [0.315], 'number': [0.274], 'incremented': [0.267], 'build': [0.191]}, {'user': [0.477], 'registry': [0.338], 'project': [0.251], 'setup': [0.219], 'hkcu': [0.211]}, {'connect'

[{'integer': [0.365],
  'sqlconnection': [0.325],
  'sqlstatement': [0.441],
  'strsql': [0.239],
  'varchar': [0.241]},
 {'branching': [0.467],
  'explaining': [0.231],
  'merging': [0.455],
  'tortoisesvn': [0.467],
  'tutorials': [0.409]},
 {'asp': [0.353],
  'need': [0.209],
  'net': [0.321],
  'site': [0.375],
  'sitemap': [0.18]},
 {'color': [0.266],
  'colors': [0.295],
  'pseudo': [0.303],
  'solved': [0.282],
  'stuck': [0.275]},
 {'card': [0.603],
  'cards': [0.28],
  'class': [0.237],
  'database': [0.186],
  'game': [0.244]},
 {'classes': [0.539],
  'mine': [0.193],
  'nested': [0.278],
  'programmers': [0.185],
  'video': [0.276]},
 {'consume': [0.209],
  'consumption': [0.222],
  'services': [0.327],
  've': [0.259],
  'web': [0.411]},
 {'database': [0.211],
  'given': [0.174],
  'live': [0.405],
  'server': [0.249],
  'sql': [0.371]},
 {'build': [0.191],
  'incremented': [0.267],
  'number': [0.274],
  'settings': [0.315],
  'version': [0.465]},
 {'hkcu': [0.211],
  'pro

In [ ]:
temp1=[]
temp2=[]
for i in nlst:
  temp2=list(i.keys())
  temp1.append(temp2)
print(len(temp1))

10000


In [ ]:
newdf = df_idf[['Id']].copy()
newdf=newdf.iloc[:10000]
newdf['keywords'] = temp1
#newdf.append(df[['keywords']])
#newdf = pd.concat([df_idf['Id'], df['keywords']], axis=1)
newdfex=newdf.explode('keywords')
newdfex = newdfex.set_index(['Id'])

print(newdf,newdfex)


            Id                                           keywords
0         80.0  [sqlstatement, integer, sqlconnection, varchar...
1         90.0  [tortoisesvn, branching, merging, tutorials, e...
2        120.0                    [site, asp, net, need, sitemap]
3        180.0             [pseudo, colors, solved, stuck, color]
4        260.0               [card, cards, game, class, database]
...        ...                                                ...
9995  637840.0              [uri, scheme, geo, geographic, draft]
9996  637870.0           [signal, program, queue, message, using]
9997  637900.0  [_debug, compile, condition, defined, preproce...
9998  637910.0             [ini, file, bug, sections, incomplete]
9999  637920.0           [netbeans, switch, case, using, palette]

[10000 rows x 2 columns]                keywords
Id                     
80.0       sqlstatement
80.0            integer
80.0      sqlconnection
80.0            varchar
80.0             strsql
...           

In [ ]:
#newdf.to_csv('/content/gdrive/MyDrive/ASE/RP_DATA/StackSample 10percent of Stack Overflow Q&A/5000_Tags.csv')


In [ ]:
res = {}
for d in nlst:
    res.update(d)
for d in res:
  res[d]=res[d][0]
tagkeys=list(res.keys())
tagvalues=list(res.values())
pk = []
for i in tagkeys:
    if i not in pk:
        pk.append(i)
ntagkeys=pk
pk=[]
for i in tagvalues:
    if i not in pk:
        pk.append(i)
ntagvalues=pk
print(len(ntagkeys))

8213


In [ ]:
dfn=pd.DataFrame(ntagkeys)
dfn.columns = ['Tags']

print(dfn)

               Tags
0      sqlstatement
1           integer
2     sqlconnection
3           varchar
4            strsql
...             ...
8208        atleast
8209        paarray
8210         newval
8211     geographic
8212         _debug

[8213 rows x 1 columns]
